In [1]:
from dotenv import load_dotenv
import os
def get_api_key(api_key_name):
        return os.getenv(api_key_name)

In [2]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel
from tavily import TavilyClient
from langgraph.checkpoint.memory import MemorySaver

d:\Ineuron-materials-FSDS\Gen AI\Learning-langgraph\langgraph\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
class State(TypedDict):
    destination:str
    dates:str
    plan:str
    content:str
    itinerary:str

graph_builder = StateGraph(State)


In [4]:
#model = ChatOpenAI(api_key=get_api_key("OPENAI_API_KEY"), temperature=0.3, model='gpt-3.5-turbo')
model = ChatGroq(api_key=get_api_key("GROQ_API_KEY"),model='llama-3.1-8b-instant',temperature=0.6)

In [5]:
PLAN_PROMPT= """You are an expert travel planner tasked with creating a high \
level itinerary outline.Write such an outline for the user provided \
destination and dates. Give an outline of the itinerary."""

def plan_node(State):
    messages = [
        SystemMessage(content=f"Planing for {State['destination']} on {State['dates']}"),
        HumanMessage(content=PLAN_PROMPT)
    ]
    response =model.invoke(messages)
    return {"plan": response.content}



In [6]:
class Queries(BaseModel):
    queries: List[str]


tavily = TavilyClient(api_key=get_api_key("TAVILY_API_KEY"))


RESEARCH_PLAN_PROMPT = """You are a travel planner tasked with finding events \
for a user visiting. Generate a list of search queries that will gather \
information on local events during this period such as music festivals, \
food fairs, technical conferences, or any other noteworthy events. \
Only generate 3 queries max."""

def research_plan_node(state: State):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=f"{state['destination']}\n\nHere is my \
                               dates:\n\n{state['dates']}")
    ])
    content = []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {'content': content}



In [7]:
WRITER_PROMPT = """You are a travel agent tasked with creating the best \
possible travel itinerary for the user's trip. Generate a detailed itinerary \
based on the provided destination, dates, and any relevant information. \
If the user provides feedback or changes, respond with a revised version \
of your previous itinerary.\
Utilize all the information below as needed:

{content}
"""

def generation_node(state: State):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['destination']}\n\nHere is my \
                dates:\n\n{state['dates']}\n\nHere is my \
                plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message
    ]
    response = model.invoke(messages)
    return {'itinerary': response.content}



In [8]:
# Define constants
START = "start"
END = "end"

# Safe helper functions
def add_node_safe(graph, name, node_obj):
    try:
        graph.add_node(name, node_obj)
    except Exception:
        pass

def add_edge_safe(graph, from_node, to_node):
    try:
        graph.add_edge(from_node, to_node)
    except Exception:
        pass

# Add nodes safely
add_node_safe(graph_builder, START, plan_node)
add_node_safe(graph_builder, "plan_node", plan_node)
add_node_safe(graph_builder, "research_plan", research_plan_node)
add_node_safe(graph_builder, "generation_node", generation_node)
add_node_safe(graph_builder, END, generation_node)  # if END needs a node object

# Set entry point safely
try:
    graph_builder.set_entry_point(START)
except Exception:
    pass

# Add edges safely
add_edge_safe(graph_builder, START, "plan_node")
add_edge_safe(graph_builder, "plan_node", "research_plan")
add_edge_safe(graph_builder, "research_plan", "generation_node")
add_edge_safe(graph_builder, "generation_node", END)

# Compile and display
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# from IPython.display import Image, display, Markdown
# #display(Image(graph.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))
# graph.get_graph().draw_mermaid_svg("graph.svg")
# from IPython.display import SVG, display
# display(SVG("graph.svg"))


In [9]:


thread = {'configurable': {'thread_id': '1'}}
for s in graph.stream({
    'destination': 'Jordan',
    'dates': '2024-10-21 to 2024-10-25'
}, thread):
    print(s)

{'start': {'plan': "**Jordan Itinerary: 2024-10-21 to 2024-10-25**\n\n**Day 1: October 21 - Arrival in Amman**\n\n- Arrive at Queen Alia International Airport (AMM)\n- Transfer to hotel in Amman city\n- Spend the day exploring Amman's city center, including:\n  - Citadel Hill (Amman's highest point)\n  - Roman Theatre\n  - King Hussein Mosque\n  - Traditional Jordanian markets and cafes\n\n**Day 2: October 22 - Amman to Petra**\n\n- Drive to Petra, a UNESCO World Heritage Site (approximately 4 hours)\n- Visit the Monastery (Al-Khazneh)\n- Explore the Siq (a narrow canyon)\n- Overnight in a hotel near Petra\n\n**Day 3: October 23 - Petra**\n\n- Spend a full day exploring Petra:\n  - Visit the Treasury (Al-Khazneh)\n  - Explore the Street of Facades\n  - Visit the Amphitheatre\n  - Hike to the Monastery (optional)\n- Overnight in a hotel near Petra\n\n**Day 4: October 24 - Wadi Rum and Aqaba**\n\n- Drive to Wadi Rum (approximately 1 hour), a stunning desert landscape\n- Take a 4x4 jeep t

In [10]:
snapshot = graph.get_state(thread)

def pretty_print_itinerary(itinerary):
    lines = itinerary.split('\n')
    for line in lines:
        if line.startswith('**'):
            print(f"\n{line.strip('*')}")
        elif line.startswith('-'):
            print(f"  {line}")
        else:
            print(line)

pretty_print_itinerary(snapshot.values['itinerary'])

Revised Itinerary:

Day 1: October 21, 2024
  - Arrival in New York City
  - Check into your accommodation
  - Explore Times Square and take in the bright lights and bustling atmosphere
  - Dinner at a local restaurant

Day 2: October 22, 2024
  - Visit the Statue of Liberty and Ellis Island
  - Explore the 9/11 Memorial and Museum
  - Walk around Wall Street and the Financial District
  - Dinner in Lower Manhattan

Day 3: October 23, 2024
  - Join us at Leo Bar for Asia Society Museum’s new exhibition, *Maḏayin: Eight Decades of Aboriginal Australian Bark Painting from Yirrkala*. Enjoy free museum admission, guided tours, drink specials, and music.
  - Visit Central Park and enjoy a leisurely stroll or bike ride
  - Shopping on Fifth Avenue
  - Dinner in Midtown Manhattan

Day 4: October 24, 2024
  - Food Network New York City Wine & Food Festival. Explore the culinary delights of the city with presentations from chefs and influencers.
  - Take a ferry to Staten Island for views of th

In [12]:
snapshot = graph.get_state(thread)

def pretty_print_itinerary(itinerary):
    lines = itinerary.split('\n')
    for line in lines:
        if line.startswith('**'):
            print(f"\n{line.strip('*')}")
        elif line.startswith('-'):
            print(f"  {line}")
        else:
            print(line)

pretty_print_itinerary(snapshot.values['itinerary'])


Revised Itinerary:


Day 1: Arrival in UAE (October 21, 2024)
  - Arrive in UAE and check into your hotel.
  - Relax and unwind after your journey.
  - Explore the local area and enjoy a traditional Emirati dinner.


Day 2: Dubai City Tour (October 22, 2024)
  - Visit iconic landmarks such as the Burj Khalifa, Dubai Mall, and Dubai Fountain.
  - Explore the historic Al Fahidi neighborhood and Dubai Museum.
  - Enjoy a traditional Arabic lunch.
  - Optional visit to the Dubai Marina and JBR Beach.


Day 3: Beyond Trib Fest: Dubai 2024 (October 23, 2024)
  - Attend the Beyond Trib Fest: Dubai 2024 at the Irish Village @ Garhoud, celebrating the world's greatest musical icons.
  - Enjoy a day filled with music and entertainment for the whole family.


Day 4: Desert Safari Experience (October 24, 2024)
  - Enjoy a thrilling desert safari experience with dune bashing, camel riding, and sandboarding.
  - Watch the sunset over the desert and enjoy a traditional Arabian dinner under the stars

In [14]:
snapshot = graph.get_state(thread)

def pretty_print_itinerary(itinerary):
    lines = itinerary.split('\n')
    for line in lines:
        if line.startswith('**'):
            print(f"\n{line.strip('*')}")
        elif line.startswith('-'):
            print(f"  {line}")
        else:
            print(line)

pretty_print_itinerary(snapshot.values['itinerary'])

Revised Itinerary Outline for KSA Trip on 2024-10-21 to 2024-10-25:

Day 1 (2024-10-21):
  - Arrival in Riyadh, KSA
  - Check-in at the hotel
  - Rest and relax after your journey

Day 2 (2024-10-22):
  - Breakfast at the hotel
  - Attend the InFlavour B2B F&B Trade Show at Riyadh International Convention Centre
  - Explore the various food-related exhibits and networking opportunities
  - Lunch at the event
  - Free evening to relax or explore the local area

Day 3 (2024-10-23):
  - Breakfast at the hotel
  - Attend the Intersec Saudi Arabia 2024 event at Riyadh International Convention Centre
  - Engage with the latest innovations in security, safety, and fire protection industries
  - Lunch at the event
  - Dinner at a local restaurant

Day 4 (2024-10-24):
  - Breakfast at the hotel
  - Participate in the European Food Festival at a nearby location
  - Enjoy the culinary delights and cultural experiences
  - Return to Riyadh for dinner at a traditional Saudi restaurant

Day 5 (2024-

In [10]:
snapshot = graph.get_state(thread)

def pretty_print_itinerary(itinerary):
    lines = itinerary.split('\n')
    for line in lines:
        if line.startswith('**'):
            print(f"\n{line.strip('*')}")
        elif line.startswith('-'):
            print(f"  {line}")
        else:
            print(line)

pretty_print_itinerary(snapshot.values['itinerary'])


Revised Jordan Itinerary: 2024-10-21 to 2024-10-25


Day 1: October 21 - Arrival in Amman and Festival of Taste

  - Arrive at Queen Alia International Airport (AMM)
  - Transfer to hotel in Amman city
  - Spend the day exploring Amman's city center, including:
  - Citadel Hill (Amman's highest point)
  - Roman Theatre
  - King Hussein Mosque
  - Traditional Jordanian markets and cafes
  - Attend the Festival of Taste (info provided) which is happening on October 21st, a vibrant cultural and entertainment extravaganza that showcases the rich heritage of Jordanian cuisine
  - Visit over 20 world-class cooking demonstrations, 70 restaurants, 25 food trucks, and 50 local market stalls


Day 2: October 22 - Amman to Petra and Conference

  - Drive to Petra, a UNESCO World Heritage Site (approximately 4 hours)
  - Visit the Monastery (Al-Khazneh)
  - Explore the Siq (a narrow canyon)
  - Attend the 2nd International Conference and Forum on Business and Digital Economy 2024 (IBDE) at the Un

In [1]:
import re
import spacy
from dateutil import parser as date_parser
from datetime import timedelta, date

class TravelInfo:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_md")  # good for location

    def extract_location(self, text: str):
        doc = self.nlp(text)
        locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
        return locations

    def extract_dates_and_duration(self, text: str):
        start_date, end_date, duration = None, None, None

        # Case 1: explicit date range "from X to Y"
        range_match = re.search(
            r"from\s+([\d]{1,2}\s+[A-Za-z]{3,9}\s+\d{4})\s+to\s+([\d]{1,2}\s+[A-Za-z]{3,9}\s+\d{4})",
            text,
            re.IGNORECASE,
        )
        if range_match:
            start_date = date_parser.parse(range_match.group(1)).date()
            end_date = date_parser.parse(range_match.group(2)).date()
            duration = (end_date - start_date).days
            return start_date, end_date, duration

        # Case 2: single start date with duration
        date_match = re.search(r"\b\d{1,2}\s+[A-Za-z]{3,9}\s+\d{4}\b", text)
        if date_match:
            start_date = date_parser.parse(date_match.group()).date()

        dur_match = re.search(r"(\d+)\s+days?", text.lower())
        if dur_match:
            duration = int(dur_match.group(1))

        if start_date and duration:
            end_date = start_date + timedelta(days=duration)

        return start_date, end_date, duration

    def extract_trip_info(self, text: str):
        locations = self.extract_location(text)
        start, end, trip_days = self.extract_dates_and_duration(text)

        return {
            "location": locations[0] if locations else None,
            "start_date": start.isoformat() if start else None,
            "end_date": end.isoformat() if end else None,
            "duration": trip_days,
        }


In [2]:
extractor = TravelInfo()
q = 'I want to travel Oman for 20 days from 12 December 2025'
info = extractor.extract_trip_info(q)
print(info)

{'location': 'Oman', 'start_date': '2025-12-12', 'end_date': '2026-01-01', 'duration': 20}


In [4]:
extractor = TravelInfo()
q = 'I want to travel uk for from 12 December 2025 to 30 Dec 2025'
info = extractor.extract_trip_info(q)
print(info)

{'location': None, 'start_date': '2025-12-12', 'end_date': '2025-12-30', 'duration': 18}
